In [2]:
import re
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from preprocess import get_records, preprocess_for_logistic, read_records


# read and preprocess training and validation dataset for average pool, validation dataset

In [3]:
r = re.compile("^train.+\\.tfrecord$")
train_dir  = "/Users/shufanxia/Documents/IDS705/Video_Popularity_Prediction/frame-level/"
val_dir = "/Users/shufanxia/Documents/IDS705/Video_Popularity_Prediction/validation-frame/"

frames_train = get_records(train_dir,"train")
frames_val = get_records(val_dir,"validate")

n_samples = 50
n_labels = 300
feat_rgb,feat_audio,feat_puesdoid,feat_labels = read_records(frames_train[:1])
X_rgb_train, X_audio_train,y_train = preprocess_for_logistic(feat_rgb,feat_audio,feat_labels,n_samples,n_labels)

feat_rgb_val,feat_audio_val,feat_puesdoid_val,feat_labels_val = read_records(frames_val[:1])
X_rgb_val, X_audio_val,y_val = preprocess_for_logistic(feat_rgb_val,feat_audio_val,feat_labels_val,n_samples,n_labels)

# Baseline logsitic model

In [30]:

def build_logistic_train(input_type = "rgb",bach_size=500, epoch=500, 
                    l2 = 1e-8, X_rgb_train=None,X_audio_train = None, y_train=None,
                     X_rgb_val=None, X_aduio_val= None,y_val=None):
    # build and train a one vs all multiclass classifier
    # choose from three types of inputs
    if input_type == "rgb":
        X_train = X_rgb_train
        X_val = X_rgb_val
    elif input_type == "audio":
        X_train = X_audio_train
        X_val = X_audio_val
    elif input_type == "both":
        X_train = tf.concat([X_rgb_train, X_audio_train],1)
        X_val = tf.concat([X_rgb_val, X_audio_val],1)
    else:
        print("invalid input type")
        raise ValueError

    logistic_reg = tf.keras.models.Sequential([
                tf.keras.layers.Dense(n_labels, activation='sigmoid',
                            kernel_regularizer=tf.keras.regularizers.L2(l2))
                ])

    optimizer = tf.keras.optimizers.SGD()
    logistic_reg.compile(optimizer,
                loss=tf.keras.losses.BinaryCrossentropy())

    # randomzie befor training
    indices = tf.range(start=0, limit=tf.shape(X_train)[0], dtype=tf.int32)
    idx = tf.random.shuffle(indices)
    X_train= tf.gather(X_train,idx)
    y_train = tf.gather(y_train,idx)
    # train, print training loss and validation loss after each epoch
    logistic_reg.fit(X_train,y_train,epochs=10,
                    batch_size=500,
                    validation_data = (X_val,y_val))

    return logistic_reg

##### With just rgb

In [34]:
logistic_reg = build_logistic_train(input_type = "rgb",bach_size=500, 
                            epoch=500, l2 = 1e-8,
                            X_rgb_train = X_rgb_train,y_train =y_train,  
                            X_rgb_val=X_rgb_val, y_val=y_val,)

Epoch 1/10
3/3 [==============================] - 0s 38ms/step - loss: 35.3133 - val_loss: 1.7935
Epoch 2/10
3/3 [==============================] - 0s 14ms/step - loss: 1.6762 - val_loss: 1.5516
Epoch 3/10
3/3 [==============================] - 0s 14ms/step - loss: 1.5147 - val_loss: 1.6191
Epoch 4/10
3/3 [==============================] - 0s 15ms/step - loss: 1.5241 - val_loss: 1.7683
Epoch 5/10
3/3 [==============================] - 0s 17ms/step - loss: 1.6272 - val_loss: 1.7439
Epoch 6/10
3/3 [==============================] - 0s 15ms/step - loss: 1.5887 - val_loss: 1.6082
Epoch 7/10
3/3 [==============================] - 0s 15ms/step - loss: 1.4901 - val_loss: 1.2901
Epoch 8/10
3/3 [==============================] - 0s 15ms/step - loss: 1.2342 - val_loss: 1.5765
Epoch 9/10
3/3 [==============================] - 0s 14ms/step - loss: 1.4238 - val_loss: 1.3545
Epoch 10/10
3/3 [==============================] - 0s 14ms/step - loss: 1.3274 - val_loss: 1.4990


##### With just audio

In [33]:
logistic_reg = build_logistic_train(input_type = "audio",bach_size=500, 
                            epoch=500, l2 = 1e-8,
                            X_audio_train =X_audio_train, y_train =y_train,  
                            X_aduio_val=X_audio_val,  y_val=y_val,)

Epoch 1/10
3/3 [==============================] - 0s 35ms/step - loss: 28.8738 - val_loss: 2.7455
Epoch 2/10
3/3 [==============================] - 0s 11ms/step - loss: 2.3719 - val_loss: 1.1906
Epoch 3/10
3/3 [==============================] - 0s 10ms/step - loss: 1.2624 - val_loss: 0.9688
Epoch 4/10
3/3 [==============================] - 0s 10ms/step - loss: 1.0505 - val_loss: 0.8663
Epoch 5/10
3/3 [==============================] - 0s 10ms/step - loss: 0.9462 - val_loss: 0.8218
Epoch 6/10
3/3 [==============================] - 0s 10ms/step - loss: 0.8873 - val_loss: 0.7845
Epoch 7/10
3/3 [==============================] - 0s 10ms/step - loss: 0.8505 - val_loss: 0.7573
Epoch 8/10
3/3 [==============================] - 0s 10ms/step - loss: 0.8219 - val_loss: 0.7426
Epoch 9/10
3/3 [==============================] - 0s 10ms/step - loss: 0.7988 - val_loss: 0.7314
Epoch 10/10
3/3 [==============================] - 0s 11ms/step - loss: 0.7860 - val_loss: 0.7200


#### With video + audio

In [31]:
logistic_reg = build_logistic_train(input_type = "both",bach_size=500, 
                            epoch=500, l2 = 1e-8,
                            X_rgb_train = X_rgb_train,X_audio_train =X_audio_train, y_train =y_train,  
                            X_rgb_val=X_rgb_val, X_aduio_val=X_audio_val,  y_val=y_val,)

Epoch 1/10
3/3 [==============================] - 1s 44ms/step - loss: 31.0875 - val_loss: 1.8518
Epoch 2/10
3/3 [==============================] - 0s 15ms/step - loss: 1.8006 - val_loss: 1.9166
Epoch 3/10
3/3 [==============================] - 0s 18ms/step - loss: 1.9545 - val_loss: 1.8386
Epoch 4/10
3/3 [==============================] - 0s 18ms/step - loss: 1.8075 - val_loss: 2.0121
Epoch 5/10
3/3 [==============================] - 0s 14ms/step - loss: 1.9208 - val_loss: 1.8151
Epoch 6/10
3/3 [==============================] - 0s 14ms/step - loss: 1.7579 - val_loss: 1.6504
Epoch 7/10
3/3 [==============================] - 0s 14ms/step - loss: 1.6201 - val_loss: 1.6150
Epoch 8/10
3/3 [==============================] - 0s 15ms/step - loss: 1.5451 - val_loss: 1.5659
Epoch 9/10
3/3 [==============================] - 0s 14ms/step - loss: 1.5329 - val_loss: 1.4487
Epoch 10/10
3/3 [==============================] - 0s 14ms/step - loss: 1.3874 - val_loss: 1.3195


In [21]:
s = build_logistic_train(X_audio_train,y_train, X_audio_val, y_val,input_type="rgb")

Epoch 1/10
1/1 [==============================] - 0s 204ms/step - loss: 47.7726 - val_loss: 15.5031
Epoch 2/10
1/1 [==============================] - 0s 14ms/step - loss: 15.2373 - val_loss: 3.3108
Epoch 3/10
1/1 [==============================] - 0s 15ms/step - loss: 3.2327 - val_loss: 0.6140
Epoch 4/10
1/1 [==============================] - 0s 16ms/step - loss: 0.6127 - val_loss: 0.1186
Epoch 5/10
1/1 [==============================] - 0s 15ms/step - loss: 0.1179 - val_loss: 0.0231
Epoch 6/10
1/1 [==============================] - 0s 15ms/step - loss: 6.4167e-05 - val_loss: 0.0228
Epoch 7/10
1/1 [==============================] - 0s 16ms/step - loss: 5.6401e-05 - val_loss: 0.0226
Epoch 8/10
1/1 [==============================] - 0s 16ms/step - loss: 5.0774e-05 - val_loss: 0.0223
Epoch 9/10
1/1 [==============================] - 0s 15ms/step - loss: 4.6429e-05 - val_loss: 0.0221
Epoch 10/10
1/1 [==============================] - 0s 15ms/step - loss: 4.2938e-05 - val_loss: 0.0219
